## Biterm Topic Model

### Import data

In [7]:
import os
import json

dirpath = os.getcwd() + '\\preprocessing\\tokens.json'

with open(dirpath, 'r') as f:
    documents = json.load(f)
    texts = [' '.join(doc) for doc in documents]

In [8]:
import pandas as pd

# Convert to DataFrame
df = pd.DataFrame(texts, columns=['headline_text'])
df.head()

,headline_text
0,март год полный солнечный_затмение полный фаза...
1,сентябрь полный лунный_затмение европейский ча...
2,март год полный солнечный_затмение полный фаза...
3,июль год продолжительный нынешний столетие пол...
4,первый упоминание солнечный_затмение год эра д...


### Create corpus and vocabulary

In [9]:
from gensim import corpora

# Map each token to a unique ID
dictionary = corpora.Dictionary(documents)
print(f'Number of unique tokens: {len(dictionary)}')

# Filter out tokens by frequency
min_doc, max_doc = 15, .2
dictionary.filter_extremes(no_below=min_doc, no_above=max_doc)
print(f'Number of unique tokens (filtered): {len(dictionary)}')

# Create a BOW corpus
corpus = [dictionary.doc2bow(doc) for doc in documents]
print(f'Number of documents: {len(corpus)}')

Number of unique tokens: 30281
Number of unique tokens (filtered): 3231
Number of documents: 1260


### Create a sparse matrix and vocabilary from gensim's BOW document representation

In [10]:
import numpy as np
from gensim import matutils

def bow_iterator(docs, dictionary):
    for doc in docs:
        yield dictionary.doc2bow(doc)

def get_term_matrix(msgs, dictionary):
    bow = bow_iterator(msgs, dictionary)
    X = np.transpose(matutils.corpus2csc(bow).astype(np.int64))
    return X

def get_vocab(dictionary):
    tmp = []
    for i in range(len(dictionary)):
        tmp.append(dictionary.get(i))
    return tuple(tmp)

def get_vocab_dict(dictionary):
    return {k: v for k, v in dictionary.token2id.items()}


# Obtain term frequency in a sparse matrix and corpus vocabulary
# these are used instead of the default ones
X = get_term_matrix(documents, dictionary)
vocab = get_vocab(dictionary)
vocab_dict = get_vocab_dict(dictionary)

In [11]:
import bitermplus as btm

tf = np.array(X.sum(axis=0)).ravel()

# Vectorize documents
docs_vec = btm.get_vectorized_docs(texts, vocab)
docs_lens = list(map(len, docs_vec))

# Generate biterms
biterms = btm.get_biterms(docs_vec)

### Train and run the Biterm topic model

In [12]:
num_topics = 15
top_words = 15 # number of top words for coherence calculation

# Train the model
biterm_model = btm.BTM(X, vocab, seed=12321, T=num_topics, \
                       M=top_words, alpha=50/8, beta=0.01)

# Run the model
biterm_model.fit(biterms, iterations=50)

100%|██████████| 50/50 [00:28<00:00,  1.73it/s]


In [13]:
# Calculate documents vs topics probability matrix
p_zd = biterm_model.transform(docs_vec)

100%|██████████| 1260/1260 [00:00<00:00, 3694.64it/s]


### Calculate metrics

In [14]:
perplexity = btm.perplexity(biterm_model.matrix_topics_words_, p_zd, X, num_topics)
coherence = btm.coherence(biterm_model.matrix_topics_words_, X, M=top_words)

print(f'Perplexity: {perplexity}' +
      f'\nCoherence score: {coherence}')

Perplexity: 964.6686193783468
Coherence score: [-150.82009045 -164.18126106 -146.45792799 -122.63203931 -180.36055583
 -100.99972679 -142.93398439 -114.88363067 -134.17174807 -136.1583634
 -150.50400074 -136.94536709 -189.10491261 -181.43845381 -170.12304129]


### Show most probable topic for each document

In [15]:
btm.get_docs_top_topic(texts, biterm_model.matrix_docs_topics_)

,documents,label
0,март год полный солнечный_затмение полный фаза...,3
1,сентябрь полный лунный_затмение европейский ча...,3
2,март год полный солнечный_затмение полный фаза...,3
3,июль год продолжительный нынешний столетие пол...,3
4,первый упоминание солнечный_затмение год эра д...,3
...,...,...
1255,группа физик гарвардский техасский_университет...,1
1256,иранский физика эксперимент вращение плёнка жи...,1
1257,физик новый тип сверхпроводник общий химически...,1
1258,исследователь бордо франция вихрь определённый...,1


### Show top words for each topic

In [16]:
num_words = 15
btm.get_top_topic_words(biterm_model, num_words)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14
0,позвоночный,атом,геном,луна,самец,звезда,птица,физика,порода,геном,физика,опухоль,волна,растение,мозг
1,эволюционный,электрон,днк,затмение,самка,галактика,популяция,нейтрино,атмосфера,кость,научный,мышь,капля,вымирание,нейрон
2,предок,поле,бактерия,солнце,особь,чёрный_дыра,хищник,детектор,зона,сапиенс,наука,рак,точка,океан,отбор
3,ветвь,магнитный_поле,эукариот,фаза,насекомое,излучение,окраска,ядро,углерод,находка,проект,мутация,изображение,морской,культура
4,общий_предок,сверхпроводник,фермент,телескоп,половой,ядро,генетический,протон,глубина,неандерталец,решение,ткань,колебание,атмосфера,сигнал
5,губка,сверхпроводимость,мутация,планета,сигнал,солнечный,отбор,измерение,кислород,пещера,эйнштейн,экспрессия,частота,климат,ребёнок
6,личинка,квантовый,последовательность,орбита,потомство,вселенная,растение,распад,мантия,след,профессор,рецептор,лёд,содержание,мышь
7,многоклеточный_животное,частота,растение,астероид,яйцо,солнце,численность,электрон,содержание,днк,книга,метастаз,сигнал,сообщество,социальный
8,нервный_система,сверхпроводящий,синтез,аппарат,колония,звёздный,дерево,коллаборация,железо,популяция,публикация,нормальный,измерение,район,память
9,гребневик,излучение,митохондрия,высота,муха,космический,особь,событие,океан,генетический,физический,раковый,распределение,численность,популяция


### Show top documents for each topic

In [17]:
docs_num = 5
btm.get_top_topic_docs(texts, biterm_model.matrix_docs_topics_, docs_num)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14
0,жаберный_щель уникальный признак тип хордовое ...,сверхпроводимость квантовый состояние вещество...,учёный чехия канада одноклеточный эукариотичес...,лунный_затмение февраль год полный житель евро...,комар мочь частота акустический брачный сигнал...,группа астроном новый ультрафиолетовый телеско...,пример разный популяция вид бразильский пальма...,главный коллаборация большой_адронный коллайде...,результат петролый-термомеханический моделиров...,обобщение материал собранный пещера мандрен фр...,жизнь дмитрий_борисович_зимин горестный отчаян...,группа исследователь клетка раковый_опухоль де...,определённый условие капля жидкость горячий по...,спор причина массовый_вымирание крупный животн...,сочувствие человек боль мозг область чувство с...
1,проблема происхождение уникальный трубчатый це...,японский учёный возможность использование свер...,нуклеиновый_кислота носитель генетический инфо...,мартовский затмение год восточный район европе...,устойчивый сохранение популяция разный группа ...,спектр квазар гамма-всплеск яркий объект вселе...,канадский американский зоолог певчий_птица тро...,коллаборация любопытный результат новый анализ...,газово-жидкий включение кристалл породообразую...,археологический остров юго-восточный_азия прих...,жизнь дмитрий_борисович_зимин горестный отчаян...,американский учёный роль корень рак раковый ст...,материал подверженный обледенение авиастроение...,район акватория южный_океан оконечность антарк...,американский учёный ряд поведенческий эксперим...
2,коралловый_полип древний эволюционный_ветвь гр...,группа физик сша россия германия бельгия полны...,вопрос происхождение синтез белок аппарат тран...,сентябрь полный лунный_затмение европейский ча...,конфликт эволюционный интерес самец самка секс...,интернациональный коллектив астроном астрофизи...,бейтсовский мимикрия неядовитый вид ядовитый я...,коллаборация обработка настоящий время статист...,австралийский учёный основа результат изотопны...,денисов_пещера алтай бесценный археологический...,начало неделя сми сенсационный сообщение омски...,линия лабораторный мышь формирование новый лим...,орёл решка определённый условие результат брос...,эколог шведский университет_умео биогенный эле...,нейробиолог влияние полиморфизм ген рецептор д...
3,древний эволюционный_ствол двусторонне-симметр...,последний теоретический экспериментальный иссл...,коллектив американский генетик свойство работа...,первый весенний месяц год житель земля великол...,самка африканский бабочка-нимфалида вид частый...,время вселенная ускорение галактика млечный_пу...,контрастный окраска наземный хищный млекопитаю...,прошедший март серия конференция результат бол...,гранат алмаз кимберлитовый_трубка яхерсфонтейн...,международный_коллектив палеогенетик археолог ...,декабрь окружный федеральный судья город гарри...,модель рак_молочный железа мышь американский а...,рост гигантский дюна счёт поглощение мелкий дю...,фитопланктон процесс фотосинтез органический_в...,работа помощь экономический игра альтруистичес...
4,германский палеонтолог древний головохоботный_...,маленький мочь сверхпроводник сверхпроводящий ...,эукариот результат симбиогенез архейный клетка...,июль год продолжительный нынешний столетие пол...,паук самка крупный агрессивный самец период сп...,помощь космический телескоп год вселенная ульт...,африканский птица нектарница нектар цветковый ...,рутинный поиск эффект новый_физика канал рожде...,катархейский гадейский эон год ранний период г...,денисовский человек пора известный скудный нах...,великий физика сила время детальный изучение о...,немецкий учёный новый противораковый вакцина м...,игровой вид спорт мяч воздух путь удивление иг...,дистанционный метод обследование поверхность з...,ангедония утрата способность удовольствие глав...


### Visualize the results

In [19]:
import tmplot as tmp

topics_coords = tmp.prepare_coords(biterm_model)

tmp.plot_scatter_topics(topics_coords, size_col='size', label_col='label')

C:\Users\satan\AppData\Roaming\Python\Python39\site-packages\sklearn\manifold\_t_sne.py:795: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
C:\Users\satan\AppData\Roaming\Python\Python39\site-packages\sklearn\manifold\_t_sne.py:805: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


alt.LayerChart(...)

In [20]:
phi = tmp.get_phi(biterm_model)
terms_probs = tmp.calc_terms_probs_ratio(phi, topic=5, lambda_=1)

tmp.plot_terms(terms_probs)

alt.Chart(...)